In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [4]:
tf.__version__

'2.1.0'

# Time Windows

We create time windows when dealing with time series data so that the alogorithm we use, may predict the next value given all the previous values.


In [5]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
  print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [6]:
# We will not create windows from the series above.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(),end='')
  print()

01234
12345
23456
34567
45678
56789
6789
789
89
9


In [7]:
# We see that there are 5 windows now, we will next drop the remainders.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [8]:
# Since we need tensors to feed in out model, we will create tensors from our output.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
  print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [9]:
# Lets now split the data in x and y.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1],window[-1:]))
for x,y in dataset:
  print(x.numpy(),y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [10]:
# Now lets shuffle the data that we have to eliminate any patters.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1],window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x,y in dataset:
  print(x.numpy(),y.numpy())

[3 4 5 6] [7]
[0 1 2 3] [4]
[1 2 3 4] [5]
[4 5 6 7] [8]
[2 3 4 5] [6]
[5 6 7 8] [9]


In [11]:
# Lets now batch these data and enable prefetch for parallel computing.

dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
    print("x =", x.numpy())
    print("y =", y.numpy())

x = [[2 3 4 5]
 [0 1 2 3]]
y = [[6]
 [4]]
x = [[5 6 7 8]
 [4 5 6 7]]
y = [[9]
 [8]]
x = [[3 4 5 6]
 [1 2 3 4]]
y = [[7]
 [5]]


In [0]:
# now lets create a function out of all the things we did to the series. 

def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset